---
## Prophet

#### Introduction à Prophet

Prophet est une bibliothèque de prévision de séries temporelles développée par Facebook. Elle est particulièrement adaptée aux séries avec des tendances non linéaires, des variations saisonnières et des événements exceptionnels. 

#### Pourquoi utiliser Prophet ?
- **Tendance :** Modélise les changements à long terme dans les données.
- **Saisonnalité :** Prend en compte les cycles récurrents (hebdomadaires, annuels, etc.).
- **Facilité d'utilisation :** Simple à configurer, même avec des données complexes ou irrégulières.
- **Robustesse :** Gère efficacement les valeurs manquantes et les anomalies.

Prophet est idéal pour des prévisions rapides et fiables, que ce soit pour des données commerciales, climatiques ou autres applications de séries temporelles.


In [1]:
import pandas as pd
import plotly_express as px
from prophet import Prophet
from useful_functions import *
file_path = "./results.json"

/Users/lilya/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [3]:
# --training--
vars = ['date', 'sales']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
model_prophet.add_seasonality(name='weekly', period=7, fourier_order=3)
model_prophet.add_seasonality(name='annual', period=365, fourier_order=10)

model_prophet.fit(train_data)

future1 = model_prophet.make_future_dataframe(periods=0)
forecast1 = model_prophet.predict(future1)
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
series_train['Train_Forecast_prophet'] = forecast1['yhat'].values

# --testing--
test_data = series_test[['date']].reset_index(drop=True)  
test_data.rename(columns={'date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat']
series_test['Test_Forecast_prophet'] = y_test_pred.values

23:00:25 - cmdstanpy - INFO - Chain [1] start processing
23:00:25 - cmdstanpy - INFO - Chain [1] done processing


```python
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], y_test_pred)
        
save_model_results(
    file_path=file_path,
    model_name="Prophet",
    params= 'without exogeneous variables',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [4]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()

In [5]:
fig = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet'],title="Predictions du test")
fig.show()

In [6]:
nrmse, mape, mae,r2 = calculate_metrics(series_test['sales'], y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.10316246994897302
1.3607503235952088e+20
236921.57783959268
-0.29833163140150365


/Users/lilya/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



Les pics semblent corrects, mais nous restons légèrement au-dessus.  
Les résultats du NRMSE paraissent satisfaisants.  

### Essayons d'intégrer les variables exogènes dans le model :**`onpromotion`**, **`oil_price`**, **`holiday`**.

In [7]:
# --training--
vars = ['date', 'sales','holiday','onpromotion','oil_price']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
model_prophet.add_seasonality(name='weekly', period=7, fourier_order=3)
model_prophet.add_seasonality(name='annual', period=365, fourier_order=10)

# __exogs__
model_prophet.add_regressor('holiday')
model_prophet.add_regressor('onpromotion')
model_prophet.add_regressor('oil_price')


model_prophet.fit(train_data)

future2 = model_prophet.make_future_dataframe(periods=0)
# __exogs__
future2['holiday'] = train_data['holiday']
future2['onpromotion'] = train_data['onpromotion']
future2['oil_price'] = train_data['oil_price']

forecast2 = model_prophet.predict(future2)
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
y_train_pred = forecast2['yhat']
series_train['Train_Forecast_prophet'] = y_train_pred.values

# --testing--
test_vars =['date','holiday','oil_price','onpromotion']
test_data = series_test[test_vars].reset_index(drop=True)  
test_data.rename(columns={'date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat'] 
series_test['Test_Forecast_prophet'] = y_test_pred.values

23:00:27 - cmdstanpy - INFO - Chain [1] start processing
23:00:27 - cmdstanpy - INFO - Chain [1] done processing


```python
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], y_test_pred)
        
save_model_results(
    file_path=file_path,
    model_name="Prophet_exogs",
    params='with exogenous variables',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [8]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()
fig = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet'],title="Predictions du test")
fig.show()

In [9]:
nrmse, mape, mae,r2 = calculate_metrics(series_test['sales'], y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.1298652373550999
1.5494654531997686e+20
346338.0800661918
-1.0574438990737671


/Users/lilya/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



>L'ajout de variables exogènes ne semble pas améliorer la situation.  
>Prophet semble bien capturer les pics, mais reste toujours légèrement au-dessus.


Enregistrons ce model pour l'examiner et l'expliquer

In [11]:
import joblib

joblib.dump(model_prophet, 'prophet_model_exogs.pkl')
print("Modèle enregistré dans 'prophet_model.pkl'")

Modèle enregistré dans 'prophet_model.pkl'


--- 

# Online approch

Simulons un flux de données : tout les 7 jours, nous avons une semaine de nouvelels dnnées. Prophet n'est pas specialement concu pour uen approche en ligne, alors reentrainons le model tout les 7 jours. 
Les variables exogenes n'aidant pas, on ne les prendra pas en compte

In [27]:
vars = ['date', 'sales']
chunk_size = 7  # Prévisions hebdomadaires
window_size = 365  # Taille de la fenêtre glissante (1 an)

new_train_data = series_train[vars].reset_index(drop=True)
new_train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

predictions = []
for new_data in data_stream(series_test, chunk_size):

    new_data = new_data[vars].reset_index(drop=True)
    new_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)


    new_train_data = pd.concat([new_train_data, new_data], ignore_index=True).tail(window_size)

    model_prophet = Prophet()
    model_prophet.add_seasonality(name='weekly', period=7, fourier_order=3)
    model_prophet.add_seasonality(name='annual', period=365, fourier_order=10)

    # MAJ
    model_prophet.fit(new_train_data)


    future = model_prophet.make_future_dataframe(periods=chunk_size)
    forecast = model_prophet.predict(future)


    y_test_pred = forecast.tail(chunk_size)['yhat'].values  # Prendre uniquement les périodes futures
    new_data_indices = new_data.index[:len(y_test_pred)]  # Correspondance des indices

    predictions.append(y_test_pred)




15:32:07 - cmdstanpy - INFO - Chain [1] start processing
15:32:07 - cmdstanpy - INFO - Chain [1] done processing
15:32:08 - cmdstanpy - INFO - Chain [1] start processing
15:32:08 - cmdstanpy - INFO - Chain [1] done processing
15:32:08 - cmdstanpy - INFO - Chain [1] start processing
15:32:08 - cmdstanpy - INFO - Chain [1] done processing
15:32:08 - cmdstanpy - INFO - Chain [1] start processing
15:32:09 - cmdstanpy - INFO - Chain [1] done processing
15:32:09 - cmdstanpy - INFO - Chain [1] start processing
15:32:09 - cmdstanpy - INFO - Chain [1] done processing
15:32:09 - cmdstanpy - INFO - Chain [1] start processing
15:32:09 - cmdstanpy - INFO - Chain [1] done processing
15:32:09 - cmdstanpy - INFO - Chain [1] start processing
15:32:09 - cmdstanpy - INFO - Chain [1] done processing
15:32:09 - cmdstanpy - INFO - Chain [1] start processing
15:32:09 - cmdstanpy - INFO - Chain [1] done processing
15:32:09 - cmdstanpy - INFO - Chain [1] start processing
15:32:09 - cmdstanpy - INFO - Chain [1]

In [28]:
series_test['test_Forecast_prophet_online'] = np.nan
series_test['test_Forecast_prophet_online'] = np.concatenate(predictions)[:-1]

Nos données s'arrêtent avant la fin d'une semaine complète. Par conséquent, nous avons prédit un jour de plus que les données disponibles dans le jeu de test.

In [29]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()
fig = px.line(series_test, x='date', y=['sales', 'test_Forecast_prophet_online','Test_Forecast_prophet'],title="Predictions du test")
fig.show()

> ### Approche en ligne est visuellement meilleurs

L'approche en ligne permet une meilleure clarté visuelle. 

Nous avons simulé un **flux hebdomadaire de données** et mis en place un **réentraînement du modèle chaque semaine** pour refléter les nouvelles informations apportées par les données en ligne.


In [30]:
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], series_test['test_Forecast_prophet_online'])
print(nrmse)
print(mape)
print(mae)
print(r2)

0.06430274844726859
2.358871850378296e+19
116380.89721781657
0.495569605778459


```python
save_model_results(
    file_path=file_path,
    model_name="Prophet_online",
    params='online toutles 7 jours',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)
```

In [31]:
#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)